<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-13 12:12:17
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.38 C
-------------------
Today PnL: -1.49 L (-1.07%)
Current PnL: -31.77 L (-20.26%)
CY Booked + Current PnL: -17.03 L (-10.86%)
-------------------
Total profit:  1.41 L
Total loss:  -33.17 L
-------------------
Total Booked + Current PnL: 9.95 L (7.67%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.7%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 97.13 L (70.52%)
Deployed:  1.30 C
Current:  1.38 C
CAGR/XIRR %: 3.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.28,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,61.0,H-SC,3.67,168.0,0.17,0.73,47.54,XR,ATH,FINANCE
77,TTKPRESTIG,-2.54,-26.84,36.82,0.09,27146.0,-27050.0,73727.0,770.00,71.18,40.0,M-SC,8.79,253.0,-1.00,0.54,1.47,OX40N,NTT,DURABLES
18,COALINDIA,-2.42,0.58,18.54,19.23,28057.0,870.0,151330.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.98,XY25,ATH,MINING
36,ICICIGI,-0.17,1.52,17.55,19.34,35653.0,3040.0,203149.0,2252.93,-19.79,57.0,X-MC,3.27,71.0,0.09,1.48,17.70,X40,ATH,INSURANCE
51,MEDANTA,-1.03,-5.08,30.72,24.08,37018.0,-6449.0,120501.0,1486.00,-12.71,51.0,X-SC,11.73,91.0,-0.17,0.88,10.37,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,5.64,-43.64,140.02,35.27,89604.0,-49555.0,63994.0,80.0,-57.75,55.0,L-SC,14.36,268.0,-0.55,0.46,56.63,XR,NTT,HOTELS
83,WHIRLPOOL,1.66,-27.62,151.36,81.92,125757.0,-31713.0,83085.0,2270.0,-57.64,61.0,M-SC,30.78,218.0,-0.25,0.60,16.71,XR,NTT,DURABLES
1,ABB,1.03,7.29,35.67,45.57,100127.0,19085.0,280704.0,7934.0,-31.00,70.0,H-MC,2.95,121.0,0.19,2.04,24.65,AR,NTT,ELECTRICAL
65,SIEMENS,0.95,-14.60,46.97,25.51,74647.0,-27170.0,158925.0,4671.5,-2.45,57.0,H-LC,2.62,50.0,-0.36,1.15,17.12,AR,ATH,ELECTRICAL
43,IRCTC,0.77,-24.46,102.44,52.93,209339.0,-66155.0,204353.0,1269.0,-44.05,47.0,H-MC,12.70,120.0,-0.32,1.48,3.19,XY24,BTT,TRAVEL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BSOFT,-6.13,-26.17,104.22,50.79,108231.0,-36804.0,103849.0,831.7,-5.30,43.0,H-SC,7.50,171.0,-0.34,0.75,20.91,XR,ATH,IT
70,SYMPHONY,-5.17,-32.18,47.40,-0.03,55019.0,-55068.0,116073.0,1306.0,-45.51,45.0,M-SC,10.36,206.0,-1.00,0.84,7.26,OX40N,NTT,DURABLES
80,VALIANTORG,-2.90,-44.47,213.63,74.15,194072.0,-72760.0,90845.0,838.0,-263.44,56.0,H-SC,24.55,145.0,-0.37,0.66,16.45,XR,NTT,CHEMICALS
66,SIS,-2.86,-24.07,59.71,21.26,50337.0,-26729.0,84303.0,528.0,1989.12,48.0,H-SC,4.03,163.0,-0.53,0.61,16.22,OX40N,NTT,MISC
55,RAJESHEXPO,-2.78,-65.32,188.58,0.07,89704.0,-89609.0,47568.0,518.0,1643.69,53.0,L-SC,8.80,269.0,-1.00,0.35,18.30,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.42,0.58,18.54,19.23,28057.0,870.0,151330.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.1,21.98,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.59,3.13,101.60,107.91,154701.0,4621.0,152265.0,1641.55,-16.09,65.0,M-SC,9.96,220.0,0.03,1.11,14.55,OX40N,ATH,CABLES
66,SIS,-2.86,-24.07,59.71,21.26,50337.0,-26729.0,84303.0,528.00,1989.12,48.0,H-SC,4.03,163.0,-0.53,0.61,16.22,OX40N,NTT,MISC
77,TTKPRESTIG,-2.54,-26.84,36.82,0.09,27146.0,-27050.0,73727.0,770.00,71.18,40.0,M-SC,8.79,253.0,-1.00,0.54,1.47,OX40N,NTT,DURABLES
72,TATAELXSI,-1.74,-29.53,87.36,32.03,81157.0,-38935.0,92900.0,9161.00,-23.76,33.0,H-SC,6.18,157.0,-0.48,0.67,4.06,OX40N,NTT,IT
40,INDIGOPNTS,-2.43,-30.87,44.68,0.02,53918.0,-53882.0,120677.0,1408.00,84.15,28.0,M-SC,7.45,240.0,-1.00,0.88,4.97,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-2.42,0.58,18.54,19.23,28057.0,870.0,151330.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.98,XY25,ATH,MINING
50,MASFIN,0.28,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,61.0,H-SC,3.67,168.0,0.17,0.73,47.54,XR,ATH,FINANCE
26,FINCABLES,-0.59,3.13,101.60,107.91,154701.0,4621.0,152265.0,1641.55,-16.09,65.0,M-SC,9.96,220.0,0.03,1.11,14.55,OX40N,ATH,CABLES
1,ABB,1.03,7.29,35.67,45.57,100127.0,19085.0,280704.0,7934.00,-31.00,70.0,H-MC,2.95,121.0,0.19,2.04,24.65,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.03,7.29,35.67,45.57,100127.0,19085.0,280704.0,7934.00,-31.00,70.0,H-MC,2.95,121.0,0.19,2.04,24.65,AR,NTT,ELECTRICAL
50,MASFIN,0.28,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,61.0,H-SC,3.67,168.0,0.17,0.73,47.54,XR,ATH,FINANCE
26,FINCABLES,-0.59,3.13,101.60,107.91,154701.0,4621.0,152265.0,1641.55,-16.09,65.0,M-SC,9.96,220.0,0.03,1.11,14.55,OX40N,ATH,CABLES
39,INDIAMART,-0.01,-5.46,118.66,106.72,138358.0,-6736.0,116600.0,4810.62,-53.11,47.0,H-SC,7.62,138.0,-0.05,0.85,16.64,AR,ATH,MISC
85,ZYDUSLIFE,-1.25,-4.10,41.43,35.63,83265.0,-8597.0,200977.0,1286.17,-17.63,53.0,H-MC,5.94,119.0,-0.10,1.46,11.89,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.08,-6.66,43.83,34.24,128003.0,-20844.0,292044.0,1972.00,-27.49,20.0,X-LC,4.79,6.0,-0.16,2.12,1.07,X40,NTT,IT
33,HCLTECH,-0.76,-4.93,30.26,23.83,69595.0,-11935.0,229989.0,1908.19,-0.81,23.0,X-LC,5.33,13.0,-0.17,1.67,10.34,X40,ATH,IT
8,AWL,-2.09,-34.53,140.11,57.19,299139.0,-112624.0,213503.0,485.00,-69.12,25.0,X-MC,17.41,58.0,-0.38,1.55,0.00,XY24,NTT,FMCG
73,TCS,-1.58,-25.58,59.29,18.55,166900.0,-96740.0,281497.0,4311.59,-35.92,25.0,X-LC,2.39,3.0,-0.58,2.04,0.00,X40,ATH,IT
13,BERGEPAINT,-0.52,-18.35,48.36,21.14,89772.0,-41707.0,185632.0,680.00,-24.04,27.0,X-MC,11.08,75.0,-0.46,1.35,0.00,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,-0.26,1.31,41.84,43.70,105155.0,3259.0,251327.0,735.00,-7.04,56.0,X-MC,1.86,73.0,0.03,1.83,16.56,XY24,BTT,FMCG
73,TCS,-1.58,-25.58,59.29,18.55,166900.0,-96740.0,281497.0,4311.59,-35.92,25.0,X-LC,2.39,3.0,-0.58,2.04,0.00,X40,ATH,IT
81,VBL,-0.69,-8.58,48.41,35.67,139773.0,-27115.0,288727.0,671.64,-19.52,40.0,X-LC,2.42,5.0,-0.19,2.10,4.08,X40N,ATH,FMCG
78,UNITDSPR,0.38,4.52,15.55,20.77,38054.0,10578.0,244722.0,1644.00,-4.59,66.0,X-MC,2.52,70.0,0.28,1.78,12.25,X40N,NTT,BREWERIES
65,SIEMENS,0.95,-14.60,46.97,25.51,74647.0,-27170.0,158925.0,4671.50,-2.45,57.0,H-LC,2.62,50.0,-0.36,1.15,17.12,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.71,-11.20,32.96,18.07,52347.0,-20035.0,158820.0,35195.00,-23.91,35.0,X-MC,10.13,64.0,-0.38,1.15,0.00,X40,ATH,PHARMA
8,AWL,-2.09,-34.53,140.11,57.19,299139.0,-112624.0,213503.0,485.00,-69.12,25.0,X-MC,17.41,58.0,-0.38,1.55,0.00,XY24,NTT,FMCG
13,BERGEPAINT,-0.52,-18.35,48.36,21.14,89772.0,-41707.0,185632.0,680.00,-24.04,27.0,X-MC,11.08,75.0,-0.46,1.35,0.00,XY24,NTT,PAINTS
73,TCS,-1.58,-25.58,59.29,18.55,166900.0,-96740.0,281497.0,4311.59,-35.92,25.0,X-LC,2.39,3.0,-0.58,2.04,0.00,X40,ATH,IT
58,RELAXO,-2.28,-53.60,228.26,52.31,153735.0,-77809.0,67351.0,1176.00,-50.63,35.0,X-SC,13.85,92.0,-0.51,0.49,0.15,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.67,-30.93,106.80,42.84,47955.0,-20104.0,44902.0,424.00,-54.82,49.0,X-SC,20.01,83.0,-0.42,0.33,8.01,XY24,NTT,MISC
58,RELAXO,-2.28,-53.60,228.26,52.31,153735.0,-77809.0,67351.0,1176.00,-50.63,35.0,X-SC,13.85,92.0,-0.51,0.49,0.15,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.10,-44.18,142.23,35.20,102150.0,-56850.0,71820.0,2096.00,-5.85,42.0,X-SC,20.99,93.0,-0.56,0.52,2.06,X40,NTT,FOOTWEAR
51,MEDANTA,-1.03,-5.08,30.72,24.08,37018.0,-6449.0,120501.0,1486.00,-12.71,51.0,X-SC,11.73,91.0,-0.17,0.88,10.37,XY24,NTT,HEALTHCARE
35,HONAUT,-1.52,-22.93,85.97,43.32,107910.0,-37352.0,125520.0,58357.33,-33.73,38.0,X-SC,10.28,90.0,-0.35,0.91,1.92,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.46,-22.55,43.04,10.78,80108.0,-54194.0,186124.0,452.00,-53.82,38.0,X-LC,18.93,1.0,-0.68,1.35,2.52,X40,NTT,FMCG
75,TMPV,-0.03,-21.43,56.51,22.98,100300.0,-48407.0,177491.0,600.00,-18.19,67.0,X-LC,3.34,2.0,-0.48,1.29,13.47,XY24,NTT,AUTO
73,TCS,-1.58,-25.58,59.29,18.55,166900.0,-96740.0,281497.0,4311.59,-35.92,25.0,X-LC,2.39,3.0,-0.58,2.04,0.00,X40,ATH,IT
81,VBL,-0.69,-8.58,48.41,35.67,139773.0,-27115.0,288727.0,671.64,-19.52,40.0,X-LC,2.42,5.0,-0.19,2.10,4.08,X40N,ATH,FMCG
42,INFY,-1.08,-6.66,43.83,34.24,128003.0,-20844.0,292044.0,1972.00,-27.49,20.0,X-LC,4.79,6.0,-0.16,2.12,1.07,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.80,-32.43,96.66,32.89,50429.0,-25034.0,52172.0,1800.00,-744.47,54.0,L-MC,11.13,259.0,-0.50,0.38,43.68,XR,NTT,BANKS
67,SONACOMS,-1.19,-8.93,52.72,39.09,48570.0,-9030.0,92129.0,804.02,-28.76,62.0,M-SC,4.42,227.0,-0.19,0.67,30.33,AR,ATH,AUTO
50,MASFIN,0.28,3.18,17.82,21.57,18016.0,3120.0,101100.0,397.04,-11.02,61.0,H-SC,3.67,168.0,0.17,0.73,47.54,XR,ATH,FINANCE
79,VAIBHAVGBL,-0.12,-20.84,103.08,60.77,149151.0,-38081.0,144694.0,521.00,66.94,62.0,H-SC,1.37,158.0,-0.26,1.05,31.18,XR,NTT,JEWELLERY
18,COALINDIA,-2.42,0.58,18.54,19.23,28057.0,870.0,151330.0,484.83,18.42,41.0,L-LC,11.78,182.0,0.03,1.10,21.98,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.03,7.29,35.67,45.57,100127.0,19085.0,280704.0,7934.00,-31.00,70.0,H-MC,2.95,121.0,0.19,2.04,24.65,AR,NTT,ELECTRICAL
79,VAIBHAVGBL,-0.12,-20.84,103.08,60.77,149151.0,-38081.0,144694.0,521.00,66.94,62.0,H-SC,1.37,158.0,-0.26,1.05,31.18,XR,NTT,JEWELLERY
11,BANDHANBNK,-1.47,-19.64,139.45,92.42,312155.0,-54712.0,223847.0,400.00,57.33,69.0,H-SC,8.08,169.0,-0.18,1.63,29.37,XY24,NTT,BANKS
7,ATULAUTO,-1.59,-13.06,67.43,45.57,115571.0,-25738.0,171394.0,844.00,3716.22,68.0,M-SC,5.79,245.0,-0.22,1.24,29.71,XY24,NTT,AUTO
67,SONACOMS,-1.19,-8.93,52.72,39.09,48570.0,-9030.0,92129.0,804.02,-28.76,62.0,M-SC,4.42,227.0,-0.19,0.67,30.33,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.69
2,50,76.68


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.59
MC    31.21
LC    23.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.72
X40      23.11
X40N     15.28
AR        9.03
XR        8.85
XY25      8.72
OX40N     7.40
SR        0.88
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.47
H-SC    22.83
X-LC    19.85
M-SC    11.56
X-SC    10.39
H-MC     4.98
M-MC     1.38
H-LC     1.15
L-LC     1.10
M-LC     1.09
L-SC     0.81
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.02,-14.48,52.94
FINANCE,13.42,-12.97,57.43
IT,11.55,-34.14,101.08
MISC,7.10,-33.00,87.41
ELECTRICAL,6.25,-9.03,48.81
PAINTS,5.05,-32.96,52.71
INSURANCE,4.86,-1.40,36.10
PHARMA,4.14,-2.90,35.30
AUTO,3.20,-18.86,59.96


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3294681.0,21
AR,1328751.0,10
X40,1311977.0,15
XR,1268859.0,12
X40N,1018188.0,10
OX40N,773825.0,10
XY25,418486.0,6
SR,298134.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3648078.0,24
X-MC,1656808.0,16
M-SC,1466903.0,15
X-LC,1156791.0,12
X-SC,862591.0,9
H-MC,392731.0,3
L-SC,179308.0,2
M-LC,141501.0,1
H-LC,74647.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251188.0      6
           AR         947726.0      5
M-SC       XY24       823501.0      6
H-SC       XR         808095.0      7
X-LC       X40        598415.0      6
X-MC       X40        540760.0      7
           XY24       494066.0      3
           X40N       412142.0      4
X-SC       X40N       378892.0      4
H-SC       OX40N      342935.0      4
M-SC       OX40N      341186.0      5
X-SC       XY24       310897.0      3
H-SC       SR         298134.0      2
X-LC       X40N       227154.0      2
X-MC       XY25       209840.0      2
H-MC       XY24       209339.0      1
X-LC       XY24       205690.0      2
H-MC       AR         183392.0      2
M-SC       XR         179230.0      2
X-SC       X40        172802.0      2
M-LC       XR         141501.0      1
X-LC       XY25       125532.0      2
M-SC       AR         122986.0      2
L-SC       OX40N       89704.0      1
           XR          89604.0      1
H-LC       AR          74647.0      1
M-MC       XY25        55057.0      1
L-MC       XR          50429.0      1
L-LC       XY25        28057.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
